In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/PremierLeague.csv")
df

,MatchID,Season,MatchWeek,Date,Time,HomeTeam,AwayTeam,FullTimeHomeTeamGoals,FullTimeAwayTeamGoals,FullTimeResult,...,MarketMaxAwayTeam,MarketAvgHomeTeam,MarketAvgDraw,MarketAvgAwayTeam,MarketMaxOver2.5Goals,MarketMaxUnder2.5Goals,MarketAvgOver2.5Goals,MarketAvgUnder2.5Goals,HomeTeamPoints,AwayTeamPoints
0,1993-1994_Arsenal_Coventry,1993-1994,1,1993-08-14,NaN,Arsenal,Coventry,0,3,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
1,1993-1994_Aston Villa_QPR,1993-1994,1,1993-08-14,NaN,Aston Villa,QPR,4,1,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2,1993-1994_Chelsea_Blackburn,1993-1994,1,1993-08-14,NaN,Chelsea,Blackburn,1,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3
3,1993-1994_Liverpool_Sheffield Weds,1993-1994,1,1993-08-14,NaN,Liverpool,Sheffield Weds,2,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
4,1993-1994_Man City_Leeds,1993-1994,1,1993-08-14,NaN,Man City,Leeds,1,1,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,2024-2025_Southampton_Everton,2024-2025,10,2024-11-02,15:00,Southampton,Everton,1,0,H,...,2.75,2.65,3.48,2.66,1.86,2.06,1.83,2.02,3,0
11876,2024-2025_Wolves_Crystal Palace,2024-2025,10,2024-11-02,17:30,Wolves,Crystal Palace,2,2,D,...,2.92,2.53,3.40,2.85,1.93,2.00,1.89,1.95,1,1
11877,2024-2025_Tottenham_Aston Villa,2024-2025,10,2024-11-03,14:00,Tottenham,Aston Villa,4,1,H,...,3.71,1.93,4.07,3.55,1.47,2.90,1.44,2.78,3,0
11878,2024-2025_Man United_Chelsea,2024-2025,10,2024-11-03,16:30,Man United,Chelsea,1,1,D,...,2.61,2.58,3.78,2.56,1.47,2.93,1.44,2.79,1,1


In [7]:
pl_23_24 = df[df.Season == "2023-2024"]
pl_23_24['Date'] = pd.to_datetime(pl_23_24['Date'])

C:\Users\juanj\AppData\Local\Temp\ipykernel_11940\224177901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_23_24['Date'] = pd.to_datetime(pl_23_24['Date'])


In [9]:
odds_df = pl_23_24[["HomeTeam","AwayTeam","FullTimeResult","MarketMaxHomeTeam","MarketMaxDraw","MarketMaxAwayTeam"]]

In [12]:
odds_df

,HomeTeam,AwayTeam,FullTimeResult,MarketMaxHomeTeam,MarketMaxDraw,MarketMaxAwayTeam
11400,Burnley,Man City,A,9.50,5.68,1.39
11401,Arsenal,Nott'm Forest,H,1.21,8.50,17.50
11402,Bournemouth,West Ham,D,2.80,3.62,2.75
11403,Brighton,Luton,H,1.36,6.00,10.50
11404,Everton,Fulham,A,2.30,3.57,3.45
...,...,...,...,...,...,...
11775,Crystal Palace,Aston Villa,H,1.93,4.50,3.84
11776,Liverpool,Wolves,H,1.18,10.00,17.00
11777,Luton,Fulham,A,3.00,4.35,2.30
11778,Man City,West Ham,H,1.11,14.00,25.00


In [13]:
odds_df.FullTimeResult.value_counts()

FullTimeResult
H    175
A    123
D     82
Name: count, dtype: int64

In [39]:
# Cálculo de la rentabilidad

def calcular_rentabilidad_HDA(row, amm=10, result=""):
    if result == "H":
        if row["FullTimeResult"] == "H":
            return amm * row["MarketMaxHomeTeam"] - amm
        else:
            return -amm
    elif result == "D":
        if row["FullTimeResult"] == "D":
            return amm * row["MarketMaxDraw"] - amm
        else:
            return -amm
    elif result == "A":
        if row["FullTimeResult"] == "A":
            return amm * row["MarketMaxAwayTeam"] - amm
        else:
            return -amm
    else:
        return 0


In [ ]:
# Amm es la cantidad que se va a poner en cada apuesta
# amm = 10
# El bankroll es la cantidad total a apostar, que se calcula al multiplicar la cantidad apostada en cada partido con los partidos totales
# También lo usaremos para calcular la rentabilidad total
bankroll = len(odds_df) * 10

In [40]:
total = odds_df.apply(lambda row: calcular_rentabilidad_HDA(row, result="H"), axis=1).sum()
rentabilidad = total * 100 / bankroll
rent_home = [total,rentabilidad]

print("Total", round(total,2))
print(f"Rentabilidad: {round(rentabilidad,2)}%")

Total -54.4
Rentabilidad: -1.43%


In [41]:
total = odds_df.apply(lambda row: calcular_rentabilidad_HDA(row, result="D"), axis=1).sum()
rentabilidad = total * 100 / bankroll
rent_draw = [total,rentabilidad]

print("Total", round(total,2))
print(f"Rentabilidad: {round(rentabilidad,2)}%")

Total -188.1
Rentabilidad: -4.95%


In [42]:
total = odds_df.apply(lambda row: calcular_rentabilidad_HDA(row, result="A"), axis=1).sum()
rentabilidad = total * 100 / bankroll
rent_away = [total,rentabilidad]

print("Total", round(total,2))
print(f"Rentabilidad: {round(rentabilidad,2)}%")

Total -310.1
Rentabilidad: -8.16%


In [44]:
resultados_odds_HDA = pd.DataFrame({
    "Estrategia": ["Todo a Home", "Todo a Draw", "Todo a Away"],
    "Total(€)": [rent_home[0], rent_draw[0], rent_away[0]],
    "Rentabilidad(%)": [rent_home[1], rent_draw[1], rent_away[1]]
})

resultados_odds_HDA

,Estrategia,Total(€),Rentabilidad(%)
0,Todo a Home,-54.4,-1.431579
1,Todo a Draw,-188.1,-4.950000
2,Todo a Away,-310.1,-8.160526
